# `Table` Usage

In this notebook we'll show common `Table` operations using the stock market data introduced in the previous notebook.  `NDSparse` operations are *nearly* identical, so we will focus on `Table`.  The functions we cover here are:

1. `select`
1. `filter`
1. `map`
1. `reduce`
1. `groupreduce`
1. `groupby`
1. `summarize`
1. `columns`/`rows`
1. `join`
1. `merge`

### Documentation

Each of the following functions has detailed inline documentation, accessed from a Julia REPL with `?select`, for example.

# Begin by loading data

In [1]:
using JuliaDB

IndexedTables.set_show_compact!(false)  # Force printing

t = loadtable("stocksample", filenamecol=:Ticker, indexcols=[:Ticker, :Date])

Table with 56023 rows, 8 columns:
Ticker         Date        Open     High     Low      Close    Volume    OpenInt
────────────────────────────────────────────────────────────────────────────────
"aapl.us.txt"  1984-09-07  0.42388  0.42902  0.41874  0.42388  23220030  0
"aapl.us.txt"  1984-09-10  0.42388  0.42516  0.41366  0.42134  18022532  0
"aapl.us.txt"  1984-09-11  0.42516  0.43668  0.42516  0.42902  42498199  0
"aapl.us.txt"  1984-09-12  0.42902  0.43157  0.41618  0.41618  37125801  0
"aapl.us.txt"  1984-09-13  0.43927  0.44052  0.43927  0.43927  57822062  0
"aapl.us.txt"  1984-09-14  0.44052  0.45589  0.44052  0.44566  68847968  0
"aapl.us.txt"  1984-09-17  0.45718  0.46357  0.45718  0.45718  53755262  0
"aapl.us.txt"  1984-09-18  0.45718  0.46103  0.44052  0.44052  27136886  0
"aapl.us.txt"  1984-09-19  0.44052  0.44566  0.43157  0.43157  29641922  0
"aapl.us.txt"  1984-09-20  0.43286  0.43668  0.43286  0.43286  18453585  0
"aapl.us.txt"  1984-09-21  0.43286  0.44566  0.42388  

# Return a subset of columns: 
## `select(table, selection)`

### Note: Multiple selections are "passed around" as a `NamedTuple`

Therefore, a function applied to multiple selections must accept a `NamedTuple` as its argument.

In [2]:
select(t, (:High, :Low) => row -> row.High - row.Low)

56023-element Array{Float64,1}:
  0.01028
  0.0115 
  0.01152
  0.01539
  0.00125
  0.01537
  0.00639
  0.02051
  0.01409
  0.00382
  0.02178
  0.00641
  0.0077 
  ⋮      
  7.03   
  7.93   
  6.53   
 11.77   
 12.349  
 16.06   
 11.12   
  8.49   
  6.47   
  5.59   
  8.16   
  6.51   

# Return a subset of rows:
## `filter(fun, table; selection)` 

Return rows for which `fun` returns true.

In [55]:
filter(x -> x[1:4] == "amzn", t; select = :Ticker)

Table with 5153 rows, 8 columns:
Ticker         Date        Open     High     Low      Close    Volume    OpenInt
────────────────────────────────────────────────────────────────────────────────
"amzn.us.txt"  1997-05-16  1.97     1.98     1.71     1.73     14700000  0
"amzn.us.txt"  1997-05-19  1.76     1.77     1.62     1.71     6106800   0
"amzn.us.txt"  1997-05-20  1.73     1.75     1.64     1.64     5467200   0
"amzn.us.txt"  1997-05-21  1.64     1.65     1.38     1.43     18853200  0
"amzn.us.txt"  1997-05-22  1.44     1.45     1.31     1.4      11776800  0
"amzn.us.txt"  1997-05-23  1.41     1.52     1.33     1.5      15937200  0
"amzn.us.txt"  1997-05-27  1.51     1.65     1.46     1.58     8697600   0
"amzn.us.txt"  1997-05-28  1.62     1.64     1.53     1.53     4574400   0
"amzn.us.txt"  1997-05-29  1.54     1.54     1.48     1.51     3472800   0
"amzn.us.txt"  1997-05-30  1.5      1.51     1.48     1.5      2594400   0
"amzn.us.txt"  1997-06-02  1.51     1.53     1.5      1

# Apply a function to a selection:

## `map(fun, table; select)`

### Note: This is also achievable with `select`

In [4]:
map(first, t; select = :Ticker) == select(t, :Ticker => first)

true

With no `select` keyword argument, each row will be passed to the map function.

In [5]:
map(r -> r.High - r.Close, t)

56023-element Array{Float64,1}:
  0.00514
  0.00382
  0.00766
  0.01539
  0.00125
  0.01023
  0.00639
  0.02051
  0.01409
  0.00382
  0.01664
  0.00641
  0.0077 
  ⋮      
  4.06   
  3.72   
  3.7    
  0.42   
 11.529  
  9.43   
  0.16   
  4.72   
  0.45   
  2.58   
  1.47   
  5.37   

# `reduce`

`reduce(reducer, table; select)`

Apply `reducer` pair-wise on the selection.  For example, if a table is four rows long, `reduce(reducer, t)` is equivalent to

```julia
out = reducer(row1, row2)
out = reducer(out, row3)
out = reducer(out, row4)
```

In order to be meaningful, your `reducer` should have the associative property:

$$(A + B) + C = A + (B + C)$$

In [6]:
reduce(+, t; select = :Volume)

1830996051150

Reducing by a `NamedTuple` of selectors returns a `NamedTuple`:

In [7]:
reduce(@NT(:Volume_sum=:Volume => +, Close_sum=:Close => +), t)

(Volume_sum = 1830996051150, Close_sum = 3.9113906714000925e6)

You can also `reduce` with estimators from OnlineStats:

In [8]:
using OnlineStats

reduce(Sum(Int), t; select = :Volume)

▦ Series{0}
│ EqualWeight | nobs=56023
└── Sum{Int64}(1830996051150)

In [9]:
using OnlineStats

reduce(Mean(), t; select = :Close)

▦ Series{0}
│ EqualWeight | nobs=56023
└── Mean(69.8176)

# `groupreduce`

`groupreduce(reducer, table, by; selection)`

Apply a reducer within each group.

In [10]:
groupreduce(+, t, :Ticker; select = :Volume)

Table with 8 rows, 2 columns:
Ticker          +
────────────────────────────
"aapl.us.txt"   891950579821
"amzn.us.txt"   40385735209
"dis.us.txt"    85815802336
"googl.us.txt"  26503128932
"ibm.us.txt"    81302723803
"msft.us.txt"   634313240042
"nflx.us.txt"   62518969374
"tsla.us.txt"   8205871633

# `groupby`

`groupby(fun, table [, by]; select, flatten)`

Apply a function within each group.

In [11]:
groupby(@NT(min_max = extrema), t, :Ticker; select = :Close)

Table with 8 rows, 2 columns:
Ticker          min_max
─────────────────────────────────
"aapl.us.txt"   (0.23051, 175.61)
"amzn.us.txt"   (1.4, 1132.88)
"dis.us.txt"    (0.28298, 118.24)
"googl.us.txt"  (50.005, 1058.29)
"ibm.us.txt"    (3.3901, 186.36)
"msft.us.txt"   (0.0672, 84.56)
"nflx.us.txt"   (1.29, 202.68)
"tsla.us.txt"   (15.8, 385.0)

In [12]:
groupby(mean, t, :Ticker; select = :Volume)

Table with 8 rows, 2 columns:
Ticker          mean
─────────────────────────
"aapl.us.txt"   1.06642e8
"amzn.us.txt"   7.83732e6
"dis.us.txt"    7.10866e6
"googl.us.txt"  7.95173e6
"ibm.us.txt"    5.78297e6
"msft.us.txt"   7.9458e7
"nflx.us.txt"   1.95311e7
"tsla.us.txt"   4.41651e6

# `summarize`

`summarize(fun, table, by; select)`

Apply a function (or functions) column-wise.

In [13]:
summarize((mean, std), t, :Ticker; select = (:Open, :Close))

Table with 8 rows, 5 columns:
Ticker          Open_mean  Close_mean  Open_std  Close_std
──────────────────────────────────────────────────────────
"aapl.us.txt"   22.2844    22.281      37.7634   37.7645
"amzn.us.txt"   181.747    181.769     239.611   239.548
"dis.us.txt"    20.6162    20.6212     26.4788   26.4787
"googl.us.txt"  389.993    389.856     235.105   235.102
"ibm.us.txt"    48.5355    48.5542     49.271    49.2977
"msft.us.txt"   18.9779    18.9847     16.4161   16.424
"nflx.us.txt"   39.5034    39.5213     47.5678   47.5733
"tsla.us.txt"   150.39     150.355     107.072   107.024

# AoS and SoA

Return the columns (struct of arrays) or rows (array of structs).

`columns(t; selection)` and `rows(t; selection)`

In [14]:
# NamedTuple of Vectors
columns(t)

(Ticker = String["aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt", "aapl.us.txt"  …  "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt", "tsla.us.txt"], Date = Date[1984-09-07, 1984-09-10, 1984-09-11, 1984-09-12, 1984-09-13, 1984-09-14, 1984-09-17, 1984-09-18, 1984-09-19, 1984-09-20  …  2017-10-30, 2017-10-31, 2017-11-01, 2017-11-02, 2017-11-03, 2017-11-06, 2017-11-07, 2017-11-08, 2017-11-09, 2017-11-10], Open = [0.42388, 0.42388, 0.42516, 0.42902, 0.43927, 0.44052, 0.45718, 0.45718, 0.44052, 0.43286  …  319.18, 320.23, 332.25, 300.13, 299.5, 307.0, 301.02, 305.5, 302.5, 302.5], High = [0.42902, 0.42516, 0.43668, 0.43157, 0.44052, 0.45589, 0.46357, 0.46103, 0.44566, 0.43668  …  323.78, 331.95, 332.609, 308.69, 306.25, 307.5, 306.5, 306.89, 304.46, 308.36], Low = [0.41874, 0.41366, 0.42516, 0.41618, 0.43927, 0.44052, 0.45

In [15]:
# Vector of NamedTuples
rows(t)

56023-element IndexedTables.Columns{NamedTuples._NT_Ticker_Date_Open_High_Low_Close_Volume_OpenInt{String,Date,Float64,Float64,Float64,Float64,Int64,Int64},NamedTuples._NT_Ticker_Date_Open_High_Low_Close_Volume_OpenInt{Array{String,1},Array{Date,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Int64,1},Array{Int64,1}}}:
 (Ticker = "aapl.us.txt", Date = 1984-09-07, Open = 0.42388, High = 0.42902, Low = 0.41874, Close = 0.42388, Volume = 23220030, OpenInt = 0)
 (Ticker = "aapl.us.txt", Date = 1984-09-10, Open = 0.42388, High = 0.42516, Low = 0.41366, Close = 0.42134, Volume = 18022532, OpenInt = 0)
 (Ticker = "aapl.us.txt", Date = 1984-09-11, Open = 0.42516, High = 0.43668, Low = 0.42516, Close = 0.42902, Volume = 42498199, OpenInt = 0)
 (Ticker = "aapl.us.txt", Date = 1984-09-12, Open = 0.42902, High = 0.43157, Low = 0.41618, Close = 0.41618, Volume = 37125801, OpenInt = 0)
 (Ticker = "aapl.us.txt", Date = 1984-09-13, Open = 0.43927, High = 0.44052, Low = 0.4

# Joins

`join(left, right; how, <options>)`

Join tables together based on matching keys.

- `how`: `:inner`, `:left`, `:outer`, or`:anti`
- `<options>`: `rkey`, `lkey` (default to indexed variable), `rselect`, `lselect`

In [91]:
t1 = table(@NT(x=1:5, y = rand(5)); pkey = :x)

Table with 5 rows, 2 columns:
x  y
────────────
1  0.2506
2  0.386636
3  0.0997082
4  0.822777
5  0.838759

In [92]:
t2 = table(@NT(x=3:7, z = rand(5)); pkey = :x)

Table with 5 rows, 2 columns:
x  z
───────────
3  0.978899
4  0.242955
5  0.265189
6  0.33245
7  0.181994

In [90]:
# try :inner, :outer, :left
tjoin = join(t1, t2; how = :inner)

Table with 3 rows, 3 columns:
x  y         z
─────────────────────
3  0.324442  0.777213
4  0.419527  0.834083
5  0.796445  0.559861

# Merging

A `merge` results in a table that is still ordered by the primary key(s).

In [94]:
t1 = table(@NT(x=rand(5), y = randn(5)), pkey = :x)

Table with 5 rows, 2 columns:
x         y
───────────────────
0.428266  -0.669153
0.442316  0.554751
0.815157  1.241
0.942508  -0.437445
0.951336  1.64855

In [95]:
t2 = table(@NT(x=rand(5), y = randn(5)), pkey = :x)

Table with 5 rows, 2 columns:
x         y
────────────────────
0.306015  -0.875444
0.393352  -0.567882
0.798625  -1.06793
0.969845  -0.0149464
0.986298  -0.493919

In [96]:
merge(t1, t2)

Table with 10 rows, 2 columns:
x         y
────────────────────
0.306015  -0.875444
0.393352  -0.567882
0.428266  -0.669153
0.442316  0.554751
0.798625  -1.06793
0.815157  1.241
0.942508  -0.437445
0.951336  1.64855
0.969845  -0.0149464
0.986298  -0.493919